A factory wants to determine how many units of Product A and Product B to produce to maximize profit, given constraints on labor hours and material availability

In [1]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, value

In [3]:
# Create a synthetic dataset
data = {
    'Product': ['A', 'B'],
    'Profit': [20, 30],
    'Labor_hours': [4, 6],
    'Material_units': [3, 5]
}

df = pd.DataFrame(data)
df

,Product,Profit,Labor_hours,Material_units
0,A,20,4,3
1,B,30,6,5


In [4]:
# Initialize the model
model = LpProblem("Maximize_Profit", LpMaximize)

# Decision variables
x = LpVariable("Product_A", lowBound=0, cat='Integer')  # Units of Product A
y = LpVariable("Product_B", lowBound=0, cat='Integer')  # Units of Product B

# Extract coefficients from the dataset
profit_A = df.loc[df['Product'] == 'A', 'Profit'].values[0]
profit_B = df.loc[df['Product'] == 'B', 'Profit'].values[0]
labor_A = df.loc[df['Product'] == 'A', 'Labor_hours'].values[0]
labor_B = df.loc[df['Product'] == 'B', 'Labor_hours'].values[0]
material_A = df.loc[df['Product'] == 'A', 'Material_units'].values[0]
material_B = df.loc[df['Product'] == 'B', 'Material_units'].values[0]

# Total resources available
total_labor = 240  # hours
total_material = 200  # units

# Objective Function
model += profit_A * x + profit_B * y, "Total_Profit"

# Constraints
model += labor_A * x + labor_B * y <= total_labor, "Labor_Constraint"
model += material_A * x + material_B * y <= total_material, "Material_Constraint"


In [8]:
from pulp import LpStatus, value

model.solve()

print(f"Status: {model.status}, {LpStatus[model.status]}")
print(f"Produce {x.varValue} units of Product A")
print(f"Produce {y.varValue} units of Product B")
print(f"Total Profit: ₹{value(model.objective)}")


Status: 1, Optimal
Produce 0.0 units of Product A
Produce 40.0 units of Product B
Total Profit: ₹1200.0


In [7]:
# Create a summary dataframe
result = pd.DataFrame({
    'Product': ['A', 'B'],
    'Units_Produced': [x.varValue, y.varValue],
    'Profit_per_Unit': [profit_A, profit_B],
    'Total_Profit': [x.varValue * profit_A, y.varValue * profit_B]
})

result['Total_Profit'] = result['Total_Profit'].round(2)
result

,Product,Units_Produced,Profit_per_Unit,Total_Profit
0,A,0.0,20,0.0
1,B,40.0,30,1200.0


In [ ]:
df.columns